In [1]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [8]:
train_data=torchvision.datasets.CIFAR10("./dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader=DataLoader(train_data,batch_size=10)

In [16]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置PyTorch的缓存目录
torch.hub.set_dir('./my_torch_hub')
vgg16_false=torchvision.models.vgg16(pretrained=False)
vgg16_true=torchvision.models.vgg16(pretrained=True)

vgg16_true.classifier.add_module("add_linear",nn.Linear(1000,10))
vgg16_true.to(device)
# vgg16_true.classifier[6]=nn.Linear(4096,10)

loss=nn.CrossEntropyLoss()
optim=torch.optim.SGD(vgg16_true.parameters(),lr=0.01)
print(vgg16_true)

cuda


D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
D:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
writer=SummaryWriter('./tensorboard/vgg16_1')
for epoch in range(20):
    running_loss=0.0
    for step,(imgs,targets) in enumerate(dataloader):
        # 将数据移动到GPU
        imgs = imgs.to(device)
        targets = targets.to(device)
        # 前向传播
        outputs=vgg16_true(imgs)
        result_loss=loss(outputs,targets)
        # 反向传播
        optim.zero_grad()
        result_loss.backward()
        optim.step()
        running_loss +=result_loss.item ()
        # 绘图
        writer.add_scalar("batch_loss",result_loss,epoch*len(dataloader)+step)
    print(f"Epoch {epoch+1}, Loss: {running_loss}")
    writer.add_scalar("epoch_loss",running_loss,epoch)

Epoch 1, Loss: 1257.50049290061
Epoch 2, Loss: 733.3499463386834
Epoch 3, Loss: 495.72008411679417
Epoch 4, Loss: 352.92105236806674
Epoch 5, Loss: 273.8949332521297
Epoch 6, Loss: 219.85500666021835
Epoch 7, Loss: 154.1089211247163
Epoch 8, Loss: 135.54717327005346
Epoch 9, Loss: 114.53239096338802
Epoch 10, Loss: 92.2609398807108
Epoch 11, Loss: 76.31961593659798
Epoch 12, Loss: 61.932638952665
Epoch 13, Loss: 37.83197305768226
Epoch 14, Loss: 66.55455573986183
Epoch 15, Loss: 37.356567116776205
Epoch 16, Loss: 40.66699252403669
Epoch 17, Loss: 57.78010527278093
Epoch 18, Loss: 40.24862515065979
Epoch 19, Loss: 25.89498287220431
Epoch 20, Loss: 39.90672255793106
